In [1]:
from sklearn.linear_model import LogisticRegression 
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold, KFold
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
tc = pd.read_csv('tmp_train.csv')
len(tc)


33126

In [3]:
oof1 = pd.read_csv('b5new_2_oof.csv')
oof2 = pd.read_csv('b0new_1up_oof.csv')
oof3 = pd.read_csv('b1new_1up_oof.csv')
oof4 = pd.read_csv('b4new_1up_oof.csv')
oof5 = pd.read_csv('b3new_1upup_oof.csv')
oof6 = pd.read_csv('b2new_1_oof.csv')
oof7 = pd.read_csv('b6new_1up_oof.csv')
oof8 = pd.read_csv('mmb5_1_oof.csv')

sub1 = pd.read_csv('sub_b5new_2_.csv')
sub2 = pd.read_csv('sub_b0new_1up_.csv')
sub3 = pd.read_csv('sub_b1new_1up_.csv')
sub4 = pd.read_csv('sub_b4new_1up_.csv')
sub5 = pd.read_csv('sub_b3new_1upup_.csv')
sub6 = pd.read_csv('sub_b2new_1_.csv')
sub7 = pd.read_csv('sub_b6new_1up_.csv')
sub8 = pd.read_csv('sub_mmb5_1.csv')
print(len(oof1),len(oof2),len(sub1),len(sub1))

33126 33126 10982 10982


In [ ]:
traindata = pd.concat([oof1.target,oof2.target,oof3.target,oof4.target,oof5.target,oof6.target,oof7.target,oof8],axis=1)
# data['x1']=data.target
# data['x2']=oof2
# data['x3']=oofnew.predictions

# data.drop(columns=['target'],inplace=True)
traindata.head()

,target,target,target,target,target,target,target,0
0,0.003688,0.001139,0.001852,0.000155,0.000278,0.008895,0.000502,0.001597
1,0.000095,0.000131,0.001777,0.000215,0.000481,0.002659,0.000848,0.000270
2,0.000364,0.000434,0.001018,0.000956,0.000709,0.001181,0.001237,0.000074
3,0.000513,0.000790,0.003018,0.000189,0.000071,0.002645,0.000941,0.002221
4,0.000117,0.000147,0.001008,0.000198,0.001310,0.002854,0.000395,0.001371


In [4]:
testdata = pd.concat([sub1.target,sub2.target,sub3.target,sub4.target,sub5.target,sub6.target,sub7.target,sub8.target],axis=1)
# sub1['xx1']=sub1.target
# sub1['xx2']=sub2.target
# sub1['xx3']=subnew.target
# sub1.drop(columns=['target','image_name'],inplace=True)
testdata.head()

,target,target,target,target,target,target,target,target
0,0.000166,0.000135,0.000358,0.000108,0.000491,0.000597,0.001022,0.001561
1,0.000068,0.000075,0.000141,0.000020,0.000231,0.000417,0.000433,0.000082
2,0.000124,0.000096,0.000205,0.000133,0.000161,0.000396,0.000523,0.000072
3,0.000195,0.000077,0.000165,0.000053,0.000196,0.000466,0.000651,0.000126
4,0.002020,0.000483,0.000410,0.000216,0.000365,0.002193,0.000832,0.000572


# minmax

In [14]:
testdata.iloc[4,:].mean()

0.00088639043625

In [15]:
lk=0.002
hk=0.11
y_sub=np.zeros(len(testdata))
for i in range(len(testdata)):
  #sum=0
  countlk=0
  counthk=0
  for j in range(8):
    if testdata.iloc[i,j]<lk:
      countlk+=1
    if testdata.iloc[i,j]>hk:
      counthk+=1
    #sum+=testdata.iloc[i,j]
  if countlk==8:
    y_sub[i]=testdata.iloc[i,:].min()
  elif counthk==8:
    y_sub[i]=testdata.iloc[i,:].max()
  else:
    y_sub[i]=testdata.iloc[i,:].mean()
y_sub

array([1.07719796e-04, 2.02929040e-05, 7.15390700e-05, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [16]:
subcsv = pd.read_csv('sample_submission.csv')
subcsv.target = y_sub
subcsv.head()

,image_name,target
0,ISIC_0052060,0.000108
1,ISIC_0052349,0.000020
2,ISIC_0058510,0.000072
3,ISIC_0073313,0.000053
4,ISIC_0073502,0.000886


In [17]:
subcsv.to_csv('minmax1.csv',index=False)

# adv stack

In [ ]:

# make a prediction with a stacking ensemble
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# define dataset
#X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=47)
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble

skf = KFold(n_splits=5)
kfpred = np.zeros(len(testdata))
for fold, (idxT, idxV) in enumerate(skf.split(np.arange(15)), 1):
    train_idx = tc.loc[tc['tfrecord'].isin(idxT)].index
    val_idx = tc.loc[tc['tfrecord'].isin(idxV)].index

    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    model.fit(traindata.iloc[train_idx], tc.target.iloc[train_idx])
    # make a prediction for one example
    #data = [[2.47475454,0.40165523,1.68081787,2.88940715,0.91704519,-3.07950644,4.39961206,0.72464273,-4.86563631,-6.06338084,-1.22209949,-0.4699618,1.01222748,-0.6899355,-0.53000581,6.86966784,-3.27211075,-6.59044146,-2.21290585,-3.139579]]
    kfpred += model.predict_proba(testdata)[:,-1] 
kfpred/=skf.n_splits

In [ ]:
y_sub = kfpred
y_sub

array([0.00169132, 0.00169325, 0.00169315, ..., 0.00169411, 0.00169411,
       0.00169411])

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
roc_auc_score(tc.target,yhat)

0.9965814199647578

In [ ]:
y_sub = model.predict_proba(testdata)[:,-1] 

In [ ]:
subcsv = pd.read_csv('sample_submission.csv')
subcsv.target = y_sub
subcsv.head()

,image_name,target
0,ISIC_0052060,0.001691
1,ISIC_0052349,0.001693
2,ISIC_0058510,0.001693
3,ISIC_0073313,0.001693
4,ISIC_0073502,0.001691


In [ ]:
subcsv.to_csv('stack5.csv',index=False)

# simple stack

In [ ]:
regr = LogisticRegression() 
  
regr.fit(traindata,tc.target ) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = regr.predict_proba(traindata)[:,-1] 

In [ ]:
y_pred

array([0.0032559 , 0.00318192, 0.00319402, ..., 0.00315842, 0.00315256,
       0.00315256])

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
roc_auc_score(tc.target,y_pred)

0.9844629921879671

In [ ]:
y_sub = regr.predict_proba(testdata)[:,-1] 

In [ ]:
subcsv = pd.read_csv('sample_submission.csv')
subcsv.target = y_sub
subcsv.head()

,image_name,target
0,ISIC_0052060,0.003161
1,ISIC_0052349,0.003162
2,ISIC_0058510,0.003165
3,ISIC_0073313,0.003166
4,ISIC_0073502,0.003206


In [ ]:
subcsv.to_csv('stack3.csv',index=False)

# arb

In [ ]:
oofnew = pd.read_csv('/content/oof_maybe_0.925or0.937.csv')
len(oofnew)

33126

In [ ]:
subnew = pd.read_csv('/content/sub_maybe_0.925or0.937.csv')
len(subnew)

10982

In [ ]:
oofnew.head()

,image_name,target,predictions
0,ISIC_0388873,0.0,0.089990
1,ISIC_1428473,0.0,0.290914
2,ISIC_5203791,0.0,0.169361
3,ISIC_7380175,0.0,0.149862
4,ISIC_7906462,0.0,0.186506


# mean

In [ ]:
sub1 = pd.read_csv('sub_b5new_2_.csv')
sub2 = pd.read_csv('sub_b0new_1up_.csv')
sub3 = pd.read_csv('sub_b1new_1up_.csv')
sub4 = pd.read_csv('sub_b4new_1up_.csv')
sub5 = pd.read_csv('sub_b3new_1upup_.csv')
sub6 = pd.read_csv('sub_b2new_1_.csv')
sub7 = pd.read_csv('sub_b6new_1up_.csv')
subcsv = pd.read_csv('sample_submission.csv')

In [ ]:
subcsv.target = sub1.target + sub2.target +sub3.target+sub4.target+sub5.target+sub6.target
subcsv.target /= 6
subcsv.head()

,image_name,target
0,ISIC_0052060,0.000309
1,ISIC_0052349,0.000159
2,ISIC_0058510,0.000186
3,ISIC_0073313,0.000192
4,ISIC_0073502,0.000948


In [ ]:
subcsv.to_csv('mean_ensemble_b0-b5_256.csv',index=False)